In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

In [8]:
chat = ChatOpenAI(
    temperature=0.2,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

In [6]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [4]:
# template version

example_template = PromptTemplate.from_template(
    "Human: {question}\nAI: {answer}"
)

In [5]:
example_selector = LengthBasedExampleSelector(
    example_prompt=example_template,
    examples=examples,
    max_length=1000
)

In [6]:
prompt = FewShotPromptTemplate(
    example_prompt=example_template,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Italy?\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Greece?\nAI: \n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [11]:
# chat version

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_template = ChatPromptTemplate.from_messages(
    messages=[
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}")
    ]
)

example_few_shot_template = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_template
)

prompt = ChatPromptTemplate.from_messages(
    messages=[
        ("system", "You are a geography expert. You can give a long answer"),
        example_few_shot_template,
        ("human", "What do you know about {country}?")
    ]
)

chain = prompt | chat

chain.invoke({"country": "Thailand"})

Thailand is a country located in Southeast Asia known for its vibrant culture, stunning beaches, and delicious cuisine. Here are some key facts about Thailand:

Capital: Bangkok
Language: Thai
Currency: Thai Baht
Famous for: Temples, tropical islands, spicy and flavorful cuisine, traditional Thai massages, and vibrant street markets
Tourist attractions: The Grand Palace, Wat Pho (Temple of the Reclining Buddha), Phi Phi Islands, Chiang Mai Night Bazaar, and Ayutthaya Historical Park
Climate: Tropical climate with a rainy season from May to October and a dry season from November to April
Cultural practices: Buddhism is the predominant religion, and Thai people are known for their hospitality and respect for traditions
Wildlife: Thailand is home to diverse wildlife, including elephants, tigers, and various species of monkeys
Economy: Tourism is a major industry in Thailand, along with agriculture, manufacturing, and exports of products such as rice, textiles, and electronics

Overall, Th

AIMessageChunk(content='Thailand is a country located in Southeast Asia known for its vibrant culture, stunning beaches, and delicious cuisine. Here are some key facts about Thailand:\n\nCapital: Bangkok\nLanguage: Thai\nCurrency: Thai Baht\nFamous for: Temples, tropical islands, spicy and flavorful cuisine, traditional Thai massages, and vibrant street markets\nTourist attractions: The Grand Palace, Wat Pho (Temple of the Reclining Buddha), Phi Phi Islands, Chiang Mai Night Bazaar, and Ayutthaya Historical Park\nClimate: Tropical climate with a rainy season from May to October and a dry season from November to April\nCultural practices: Buddhism is the predominant religion, and Thai people are known for their hospitality and respect for traditions\nWildlife: Thailand is home to diverse wildlife, including elephants, tigers, and various species of monkeys\nEconomy: Tourism is a major industry in Thailand, along with agriculture, manufacturing, and exports of products such as rice, text

In [12]:
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
        
    def add_example(self, example):
        self.examples.append(example)    
        
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

In [13]:
example_selector = RandomExampleSelector(
    examples=examples,
)

In [14]:
prompt = FewShotPromptTemplate(
    example_prompt=example_template,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Brazil")

'Human: What do you know about Greece?\nAI: \n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'